# TEP: A Three Node Fully Connected Network

In [84]:
#=============================================================================#
# Transmission Expansion Planning (TEP) Problem: 
# A Three Node Fully Connected Network
# Author: Sergio Lopez Banos
# GitHub: /LopezBanos
# Citation: If you use this code in your project please cite the following 
# thesis:
'''
@Thesis{SerAlOr2023,
    author      = {Sergio López Baños},
    title       = {Transmission Expansion Planning by Quantum Annealing},
    type        = {diplomathesis}, 
    institution = {Nebrija University},
    year        = {2023},
}
'''
#=============================================================================#


'\n@Thesis{SerAlOr2023,\n    author      = {Sergio López Baños},\n    title       = {Transmission Expansion Planning by Quantum Annealing},\n    type        = {diplomathesis}, \n    institution = {Nebrija University},\n    year        = {2023},\n}\n'

In [85]:
#=============================================================================#
#                            Importing packages                               #
#=============================================================================#
from itertools import permutations
import dimod
import neal
import numpy as np     
import pandas as pd    
from dimod import ConstrainedQuadraticModel, Binary, Integer, quicksum
from dwave.system import LeapHybridCQMSampler, DWaveSampler, EmbeddingComposite 

In [86]:
#=============================================================================#
#                            Fill up the coordinates                          #
#=============================================================================#      
C_iv = np.array([10, 20, 30], dtype=int)     # Investment Cost Coefficients
C_oc = np.array([10, 5, 2], dtype=int)       # Operational Cost Coefficients
C_sh = np.array([100, 100, 100], dtype=int)  # Load shedding Cost
g_max = np.array([10, 10, 10], dtype=int)    # Generator nominal power 
f_max_1 = 10                                 # Maximum power flow of transmission lines
S =  int(np.shape(g_max)[0])                 # Number of rows of the matrix
D = np.array([0, 12, 0], dtype=int)          # Demand at each node 


In [87]:
def pop_index(index, size):
    """
    Remove an index from a list.
    """
    relaxed_list = [i for i in range(size)]
    relaxed_list.remove(index)
    return relaxed_list  

# Create CQM

The following class has methods for building the CQM and running it on a D-Wave Hyrbid solver. It returns the samples and energies.

It takes as input the coefficients of the following combinatorial optimization problem:

In [88]:
class CQM:
    def __init__(self, size, C_iv, C_oc, C_sh, g_max, f_max_1, D):
        self.size = size
        self.C_iv = C_iv
        self.C_oc = C_oc
        self.C_sh = C_sh
        self.g_max = g_max
        self.f_max_1 = f_max_1
        self.D = D
      
    def build_cqm(self):
        S = self.size
        C_iv = self.C_iv
        C_oc = self.C_oc
        C_sh = self.C_sh
        g_max = self.g_max
        f_max_1 = self.f_max_1
        D = self.D

        #----------------------------------------------------------#
        # Creating the binary variables required for the CQM model #
        #----------------------------------------------------------#
        # Transmission Lines
        X = []
        for i in range(S):
            _temp = []
            for j in range(S):
                    if i!=j:
                        _temp.append(Binary('X_{}{}'.format(i+1, j+1)))
            X.append(_temp)

        x = []
        for i in range(S):
            for j in range(S):
                    if i!=j and i<j:
                        x.append(Binary('X_{}{}'.format(i+1, j+1)))
        
        # Generator INTEGER variables 
        G = [Integer('g_{}'.format(i+1), upper_bound=g_max[i], lower_bound=0) for i in range(len(g_max))] 
        
        # Load shedding INTEGER variables
        R = [Integer('r_{}'.format(i+1), upper_bound=D[i], lower_bound=0) for i in range(len(g_max))]

        # POWER FLOW OF CANDIDATE LINES
        F_in = []
        F_out = []
        for i in range(S):
            _temp_in = []
            _temp_out = []
            for j in range(S):
                if i!=j:
                    # Incoming Flow and Outcoming Flows
                    _temp_in.append(Integer('fin_{}_{}'.format(i+1, j+1), upper_bound=f_max_1, lower_bound=0))
                    _temp_out.append(Integer('fin_{}_{}'.format(j+1, i+1), upper_bound=f_max_1, lower_bound=0))

            F_in.append(_temp_in)
            F_out.append(_temp_out)
        #=============================================================================#
        #                               Build the CQM model                           #
        #=============================================================================# 
        cqm = ConstrainedQuadraticModel()
        # Objective Function
        obj_weight_value = 1

        # Investment Cost
        investment_objective = obj_weight_value * quicksum(C_iv[i] * x[i] 
                                                    for i in range(S))

        # Operational cost
        operational_objective = obj_weight_value * quicksum(C_oc[i]*G[i] 
                                                    for i in range(S))

        # Load Shedding Cost 
        shedding_objective = obj_weight_value * quicksum(C_sh[i]*R[i] 
                                                    for i in range(S))

        # Total Objective
        objective = investment_objective + operational_objective + shedding_objective

        # Add the objective to the CQM
        cqm.set_objective(objective)

        # Add constraints 
        for i in range(S):
            cross_index  = pop_index(i, S)
            incoming_flows = []
            outcoming_flows = []
            for j in range(S-1):
                incoming_flows.append(F_in[i][j])
                outcoming_flows.append(F_out[i][j])
            cqm.add_constraint(G[i] + R[i] + sum(incoming_flows)                    
                                       - sum(outcoming_flows)
                                    == D[i], label = 'Demand_{}'.format(i+1))

        # Power flow constraint of candidate lines (If the line is not build then the allowed power flow is zero)
        for line in range(S): # This can be improved with a loop
            for j in range(S-1):
                # Incoming Power Flow
                cqm.add_constraint(F_in[line][j] - f_max_1*X[line][j] <= 0)  
                # Outcoming Power Flow
                cqm.add_constraint(F_out[line][j] - f_max_1*X[line][j] <= 0)  
        return cqm
    
    def run_sampler(self):
        API_TOKEN = "DEV-35f1dde7d08863882634b316655fb49b7a769fd2"   # Use your own token
        sampler_hybrid = LeapHybridCQMSampler(token = API_TOKEN)
        cqm = self.build_cqm()
        results_hybrid = sampler_hybrid.sample_cqm(cqm, label = "ThreeNode")
        results_hybrid_feasible = results_hybrid.filter(lambda row: row.is_feasible);
        samples = []
        energies = []
        num_occurrences = []
        for sample, energy, num_occurrence in results_hybrid_feasible.data(fields=['sample', 'energy', 'num_occurrences']):
            samples.append(sample)
            energies.append(energy)
            num_occurrences.append(num_occurrence)

        return samples, energies, num_occurrences

## Run the hybrid sampler from D-Wave

In [89]:
samples, energies, num_ocurrences = CQM(S, C_iv, C_oc, C_sh, g_max, f_max_1, D).run_sampler()

In [24]:
# Group the list of dictionaries into a single dictionary
from collections import defaultdict

dd = defaultdict(list)

for d in samples: # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)

In [25]:
df = pd.DataFrame(dd) 
df['Prices'] = energies 

In [26]:
df['num_ocurrences'] = num_ocurrences

In [29]:
max(num_ocurrences)

1

In [35]:
samples[0]

{'X_12': 0.0,
 'X_13': 0.0,
 'X_23': 1.0,
 'g_1': 0.0,
 'g_2': 2.0,
 'g_3': 10.0,
 'r_1': -0.0,
 'r_2': -0.0,
 'r_3': -0.0}

In [ ]:
def list_of_dicc_to_pandas_df(list_of_dicc):
    for dicc in list_of_dicc:

In [ ]:
        # Add constraints 
        i=0
        cross_index  = pop_index(i, S)
        cqm.add_constraint(G[i] + R[i] + F_in[i][0] + F_in[i][1]                    # This can be improved with a loop
                                       - F_out[i][0] - F_out[i][1]
                                    == D[i], label = 'Demand_{}'.format(i+1))
        i=1 
        cqm.add_constraint(G[i] + R[i] + F_in[i][0] + F_in[i][1]                    # This can be improved with a loop
                                       - F_out[i][0] - F_out[i][1]
                                    == D[i], label = 'Demand_{}'.format(i+1))
        
        i=2 
        cqm.add_constraint(G[i] + R[i] + F_in[i][0] + F_in[i][1]                    # This can be improved with a loop
                                       - F_out[i][0] - F_out[i][1]
                                    == D[i], label = 'Demand_{}'.format(i+1))

In [16]:
X = []
for i in range(S):
    _temp = []
    for j in range(S):
            if i!=j and i<j:
                _temp.append(Binary('X_{}{}'.format(i+1, j+1)))
    if len(_temp) != 0:
        X.append(_temp)
X

[[BinaryQuadraticModel({'X_12': 1.0}, {}, 0.0, 'BINARY'),
  BinaryQuadraticModel({'X_13': 1.0}, {}, 0.0, 'BINARY')],
 [BinaryQuadraticModel({'X_23': 1.0}, {}, 0.0, 'BINARY')]]

In [13]:
for i in range(S):
    for j in range(S-1):
        print(i,j)

0 0
0 1
1 0
1 1
2 0
2 1
